In [19]:
%matplotlib notebook
import numpy as np
from scipy.sparse import csc_matrix, coo_matrix
from scipy.sparse.linalg import spsolve
import trimesh
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pandas as pd


In [78]:
def generate_elasticity_mat(youngs, poissons):
    E = youngs
    v = poissons
    mat = np.array([[1 - v, v, v, 0, 0, 0],
                    [v, 1 - v, v, 0, 0, 0],
                    [v, v, 1 - v, 0, 0, 0],
                    [0, 0, 0, 1 - 2*v, 0, 0],
                    [0, 0, 0, 0, 1 - 2*v, 0],
                    [0, 0, 0, 0, 0, 1 - 2*v],
                   ])
    elasticity_mat = E / ((1 + v) * (1 - 2 * v)) * mat
    return elasticity_mat

In [86]:
def add_local_stiffness(D, K_g, nodes, el_nodes):
    # https://academic.csuohio.edu/duffy_s/CVE_512_12.pdf
    x = np.array([nodes[el_nodes[0]][0], nodes[el_nodes[1]][0], nodes[el_nodes[2]][0], nodes[el_nodes[3]][0]])
    y = np.array([nodes[el_nodes[0]][1], nodes[el_nodes[1]][1], nodes[el_nodes[2]][1], nodes[el_nodes[3]][1]])
    z = np.array([nodes[el_nodes[0]][2], nodes[el_nodes[1]][2], nodes[el_nodes[2]][2], nodes[el_nodes[3]][2]])
    ones = np.array([1.0, 1.0, 1.0, 1.0])
    
    C = np.vstack((ones.T, x.T, y.T, z.T))
    #print("C: ", pd.DataFrame(C))
    IC = np.linalg.inv(C)
    B = np.zeros((6, 12))
    
    for i in range(4):
        B[0, i*3] = IC[1, i]
        B[1, i*3 + 1] = IC[2, i]
        B[2, i*3 + 2] = IC[3, i]
        
        B[3, i*3] = IC[2, i]
        B[3, i*3 + 1] = IC[1, i]
        
        B[4, i*3 + 1] = IC[3, i]
        B[4, i*3 + 2] = IC[2, i]
        
        B[5, i*3] = IC[3, i]
        B[5, i*3 + 2] = IC[1, i]
        
    B = B / (np.linalg.det(C))
    
    # TODO this is cheating, we should index the faces correctly, dunno if this will work!
    tet_volume = 1 / 6. * np.abs(np.linalg.det(C))
    K_loc_unscaled = np.dot(np.dot(B.T, D.T), B)
    K_l = K_loc_unscaled * tet_volume
    #print("tet vvolume: ", tet_volume)

    for i in range(4):
        for j in range(4):
            K_g.append([int(3 * el_nodes[i] + 0), int(3 * el_nodes[j] + 0), K_l[3 * i + 0, 3 * j + 0]])
            K_g.append([int(3 * el_nodes[i] + 0), int(3 * el_nodes[j] + 1), K_l[3 * i + 0, 3 * j + 1]])
            K_g.append([int(3 * el_nodes[i] + 0), int(3 * el_nodes[j] + 2), K_l[3 * i + 0, 3 * j + 2]])

            K_g.append([int(3 * el_nodes[i] + 1), int(3 * el_nodes[j] + 0), K_l[3 * i + 1, 3 * j + 0]])
            K_g.append([int(3 * el_nodes[i] + 1), int(3 * el_nodes[j] + 1), K_l[3 * i + 1, 3 * j + 1]])
            K_g.append([int(3 * el_nodes[i] + 1), int(3 * el_nodes[j] + 2), K_l[3 * i + 1, 3 * j + 2]])

            K_g.append([int(3 * el_nodes[i] + 2), int(3 * el_nodes[j] + 0), K_l[3 * i + 2, 3 * j + 0]])
            K_g.append([int(3 * el_nodes[i] + 2), int(3 * el_nodes[j] + 1), K_l[3 * i + 2, 3 * j + 1]])
            K_g.append([int(3 * el_nodes[i] + 2), int(3 * el_nodes[j] + 2), K_l[3 * i + 2, 3 * j + 2]])
    
    return B


# TODO, FIX THIS!!!, need to set multiple constraints
def apply_constraints(K_g_sp, constraints):
    # constraint is of type [(node, [x, y, z]), ...] where xyz are 1 for constraint, 0 for no constraint
    for constraint in constraints:
        for i in range(len(constraint[1])):
            if constraint[1][i] == 1:
                idx = constraint[0]*3 + i
                K_g_sp[idx, :] = 0.0
                K_g_sp[:, idx] = 0.0
        for i in range(len(constraint[1])):
            if constraint[1][i] == 1:
                idx = constraint[0]*3 + i
                K_g_sp[idx, idx] = 1.0
                
def get_loads(loads, verts):
    # loads is of tpye [(node, x, y, z)]
    load_vec = np.zeros(len(verts)*3)
    for load in loads:
        node = load[0]
        load_vec[3*node + 0] = load[1]
        load_vec[3*node + 1] = load[2]
        load_vec[3*node + 2] = load[3]
    return load_vec
        
def solve_fem(K_g_sp, loads):
    d = spsolve(K_g_sp, loads)
    return d

In [88]:
test_nodes = [[0, 2, 1, 3],
              [0, 1, 2, 4]]
positions = [[1., 1., 2.],
             [2., 0., 2.],
             [0., 0., 2.],
             [1., 0.5, 0.],
             [1., 0.5, 4.]]

poisson = 0.3
youngs = 2000
D = generate_elasticity_mat(youngs, poisson)

constraints = [[0, [1, 1, 1]],
               [1, [1, 1, 1]],
               [3, [1, 1, 1]]]

loads = [[4, 0., 0.1, 0]]

#print(pd.DataFrame(D))

vert_array = np.array(positions)
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(vert_array[:,0], vert_array[:, 1], vert_array[:, 2])
#plt.show()

load_arr = get_loads(loads, positions)
K_g = []
for el in test_nodes:
    add_local_stiffness(D, K_g, positions, el)
data = [k[2] for k in K_g]
rows = [k[0] for k in K_g]
cols = [k[1] for k in K_g]
K_g_sp = coo_matrix((data, (rows, cols))).tocsc()

apply_constraints(K_g_sp, constraints)

#print("Loads: ", pd.DataFrame(K_g_sp.todense()))

denseboi = K_g_sp.todense()
displacements = np.linalg.solve(denseboi, load_arr)
#displacements = solve_fem(K_g_sp, load_arr)


print(pd.DataFrame(displacements))


           0
0   0.000000
1   0.000000
2   0.000000
3   0.000000
4   0.000000
5   0.000000
6  -0.000931
7  -0.001201
8   0.002853
9   0.000000
10  0.000000
11  0.000000
12  0.003036
13  0.008972
14  0.001188


/home/heidt/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
